In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import gcsfs
import networkx as nx
from itertools import product
import pandas as pd
from joblib import Parallel, delayed

import numpy as np
import scanpy as sc
import gcsfs
import networkx as nx
import mygene
from scipy.stats import ranksums
import random


from scipy.stats import ranksums
from itertools import combinations
import random



In [2]:
# ----------------------------
# Step 1: 加载并处理 STRING PPI 数据
# ----------------------------

# 1.1 加载核心PPI网络 (protein-protein links)
# 请将 'path/to/' 替换为你实际的文件路径
ppi_file = '9606.protein.links.v12.0.txt'
ppi_df = pd.read_csv(ppi_file, sep=' ')

ppi_df

,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000356607,173
1,9606.ENSP00000000233,9606.ENSP00000427567,154
2,9606.ENSP00000000233,9606.ENSP00000253413,151
3,9606.ENSP00000000233,9606.ENSP00000493357,471
4,9606.ENSP00000000233,9606.ENSP00000324127,201
...,...,...,...
13715399,9606.ENSP00000501317,9606.ENSP00000475489,195
13715400,9606.ENSP00000501317,9606.ENSP00000370447,158
13715401,9606.ENSP00000501317,9606.ENSP00000312272,226
13715402,9606.ENSP00000501317,9606.ENSP00000402092,169


In [3]:
# 1.2 加载蛋白别名映射表 (protein to gene symbol)
aliases_file = '9606.protein.aliases.v12.0.txt'
aliases_df = pd.read_csv(aliases_file, sep='\t')

aliases_df

,#string_protein_id,alias,source
0,9606.ENSP00000000233,2B6H,Ensembl_PDB
1,9606.ENSP00000000233,2B6H,UniProt_DR_PDB
2,9606.ENSP00000000233,381,Ensembl_HGNC_entrez_id
3,9606.ENSP00000000233,381,KEGG_GENEID
4,9606.ENSP00000000233,381,KEGG_KEGGID_SHORT
...,...,...,...
3889202,9606.ENSP00000501317,regulatory factor X domain containing 2,Ensembl_HGNC_prev_name
3889203,9606.ENSP00000501317,"regulatory factor X, 7",Ensembl_HGNC_prev_name
3889204,9606.ENSP00000501317,regulatory factor X7,Ensembl_HGNC_name
3889205,9606.ENSP00000501317,uc059jng.1,Ensembl_HGNC_ucsc_id


In [4]:
ppi_df

,protein1,protein2,combined_score
0,9606.ENSP00000000233,9606.ENSP00000356607,173
1,9606.ENSP00000000233,9606.ENSP00000427567,154
2,9606.ENSP00000000233,9606.ENSP00000253413,151
3,9606.ENSP00000000233,9606.ENSP00000493357,471
4,9606.ENSP00000000233,9606.ENSP00000324127,201
...,...,...,...
13715399,9606.ENSP00000501317,9606.ENSP00000475489,195
13715400,9606.ENSP00000501317,9606.ENSP00000370447,158
13715401,9606.ENSP00000501317,9606.ENSP00000312272,226
13715402,9606.ENSP00000501317,9606.ENSP00000402092,169


In [5]:
# 1.7 (可选) 设置一个置信度阈值，只保留高置信度的相互作用
# STRING 的 combined_score 范围是 0-1000，400 通常被认为是中等置信度
confidence_threshold = 700
ppi_df = ppi_df[ppi_df['combined_score'] >= confidence_threshold].copy()

print(f"Processed PPI Network: {len(ppi_df)} edges")
ppi_df

Processed PPI Network: 473860 edges


,protein1,protein2,combined_score
85,9606.ENSP00000000233,9606.ENSP00000158762,825
130,9606.ENSP00000000233,9606.ENSP00000357048,718
160,9606.ENSP00000000233,9606.ENSP00000262305,952
197,9606.ENSP00000000233,9606.ENSP00000329419,752
268,9606.ENSP00000000233,9606.ENSP00000469035,795
...,...,...,...
13715019,9606.ENSP00000501277,9606.ENSP00000326630,942
13715034,9606.ENSP00000501277,9606.ENSP00000263726,944
13715072,9606.ENSP00000501317,9606.ENSP00000290524,780
13715299,9606.ENSP00000501317,9606.ENSP00000305071,978


In [26]:
# ----------------------------
# Step 2: 加载 Tahoe-100M 数据并计算细胞系特异性表达
# ----------------------------

# 2.1 初始化 GCS 文件系统 (用于读取 Tahoe-100M)
fs = gcsfs.GCSFileSystem()
gcp_base_path = "gs://arc-ctc-tahoe100/2025-02-25/"


In [24]:
!gcloud auth login


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=O4ZWChBP1AIWo51VyhoIbrjjueYxOg&access_type=offline&code_challenge=J-HKoxVwQ6rbMBpebXN9nXiAe1mmVUPeXP4eC8MXWII&code_challenge_method=S256


You are now logged in as [pkqpikachu@gmail.com].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [27]:
import io
import os
from tqdm import tqdm

# 2.2 加载一个 h5ad 文件
infile = gcp_base_path + "tutorial/plate3_2k-obs.h5ad"
local_file = "plate3_2k-obs.h5ad"  # 保存到当前目录，文件名同远程

fs = gcsfs.GCSFileSystem()

# 检查本地是否存在
if not os.path.exists(local_file):
    # 获取文件总大小 (字节)
    file_info = fs.info(infile)
    total_size = file_info['size']
    print(f"File size: {total_size / (1024*1024):.2f} MB")

    # 创建一个带进度条的文件对象并下载
    with fs.open(infile, 'rb') as f_raw:
        with tqdm.wrapattr(f_raw, "read", total=total_size, desc="Downloading") as f_tqdm:
            file_data = f_tqdm.read()  # 读取全部数据

        # 保存到本地
        with open(local_file, 'wb') as f_local:
            f_local.write(file_data)
    print("Download completed and saved locally.")
else:
    print("Local file found, skipping download.")

# 从本地文件加载 AnnData 对象
print("Loading AnnData object...")
adata = sc.read_h5ad(local_file)

adata

File size: 6.92 MB


Downloading: 100%|██████████| 6.92M/6.92M [00:44<00:00, 164kB/s]


Download completed and saved locally.
Loading AnnData object...


AnnData object with n_obs × n_vars = 2000 × 62710
    obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate'

In [29]:
# 2.3 数据预处理
# 清理 drug 列
adata.obs['drug'] = adata.obs['drug'].str.strip()
# 只保留高质量细胞
adata = adata[adata.obs['pass_filter'] == 'full', :]
adata

C:\Users\Keqin\AppData\Local\Temp\ipykernel_20536\3998034637.py:3: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs['drug'] = adata.obs['drug'].str.strip()


View of AnnData object with n_obs × n_vars = 1717 × 62710
    obs: 'sample', 'gene_count', 'tscp_count', 'mread_count', 'drugname_drugconc', 'drug', 'cell_line', 'sublibrary', 'BARCODE', 'pcnt_mito', 'S_score', 'G2M_score', 'phase', 'pass_filter', 'cell_name', 'plate'

In [30]:
# 2.4 获取数据集中所有唯一的细胞系
all_cell_lines = adata.obs['cell_name'].unique()
print(f"Found {len(all_cell_lines)} cell lines: {list(all_cell_lines)}")


Found 50 cell lines: ['SHP-77', 'NCI-H1792', 'LoVo', 'A498', 'PANC-1', 'CFPAC-1', 'RKO', 'HT-29', 'NCI-H2347', 'KATO III', 'HCT15', 'J82', 'NCI-H1573', 'HOP62', 'SW480', 'Panc 03.27', 'NCI-H460', 'AsPC-1', 'H4', 'HEC-1-A', 'HS-578T', 'SW 1088', 'LS 180', 'A-172', 'NCI-H2030', 'SNU-423', 'SW1417', 'C-33 A', 'RPMI-7951', 'A549', 'BT-474', 'A-427', 'NCI-H23', 'HepG2/C3A', 'SW48', 'MIA PaCa-2', 'LOX-IMVI', 'hTERT-HPNE', 'COLO 205', 'C32', 'SW 900', 'AN3 CA', 'Hs 766T', 'SNU-1', 'NCI-H596', 'SK-MEL-2', 'NCI-H2122', 'SW 1271', 'NCI-H661', 'CHP-212']


In [ ]:
# 2.5 为每个细胞系计算基因的平均表达量
# 创建一个字典来存储 {cell_line: Series of mean expression}
cell_line_expression = {}

for cell_line in all_cell_lines:
    # 筛选属于该细胞系的所有细胞
    cells_in_line = adata.obs[adata.obs['cell_name'] == cell_line].index
    if len(cells_in_line) == 0:
        continue

    # 计算平均表达量
    mean_expr = np.mean(adata[cells_in_line, :].X, axis=0).A1
    # 创建一个 Pandas Series, 索引是基因名
    expr_series = pd.Series(mean_expr, index=adata.var_names)
    expr_series_sorted = expr_series.sort_values(ascending=False)

    cell_line_expression[cell_line] = expr_series_sorted

cell_line_expression

In [32]:
# 查看 Tahoe 数据集中基因名的前20个
print("Tahoe-100M 数据集的基因命名规则 (前20个):")
print(adata.var_names[:20].tolist())

# 查看 cell_line_expression 中某个细胞系的基因名前10个，进一步确认
sample_cell_line = list(cell_line_expression.keys())[0] # 取第一个细胞系
print(f"\n细胞系 '{sample_cell_line}' 的表达数据基因名 (前10个):")
print(cell_line_expression[sample_cell_line].index[:10].tolist())

Tahoe-100M 数据集的基因命名规则 (前20个):
['TSPAN6', 'TNMD', 'DPM1', 'SCYL3', 'C1orf112', 'FGR', 'CFH', 'FUCA2', 'GCLC', 'NFYA', 'STPG1', 'NIPAL3', 'LAS1L', 'ENPP4', 'SEMA3F', 'CFTR', 'ANKIB1', 'CYP51A1', 'KRIT1', 'RAD52']

细胞系 'SHP-77' 的表达数据基因名 (前10个):
['MALAT1', 'MT-RNR2', 'MT-RNR1', 'NEAT1', 'NALF1', 'HSP90AA1', 'PDE10A', 'MT-ND4', 'EPIC1', 'ENSG00000285534']


In [36]:
# 修正：精确筛选出 Gene Symbol
# 优先选择 'Ensembl_HGNC_symbol'，如果没有，则选择 'Ensembl_HGNC'
# 创建一个优先级列表
priority_sources = ['Ensembl_HGNC_symbol', 'Ensembl_HGNC', 'BioMart_HUGO']

# 筛选高优先级的别名
high_priority_aliases = aliases_df[aliases_df['source'].isin(priority_sources)].copy()

# 创建映射字典: 取每个蛋白的第一个高优先级别名
protein_to_hgnc_symbol = high_priority_aliases.groupby('#string_protein_id')['alias'].first().to_dict()

# 将PPI网络中的蛋白ID转换为HGNC Symbol
ppi_df['gene_a'] = ppi_df['protein1'].map(protein_to_hgnc_symbol)
ppi_df['gene_b'] = ppi_df['protein2'].map(protein_to_hgnc_symbol)

# 移除映射失败的行 (即 gene_a 或 gene_b 为 NaN 的行)
ppi_df = ppi_df.dropna(subset=['gene_a', 'gene_b']).copy()

In [37]:

# 初始化 mygene 客户端
mg = mygene.MyGeneInfo()

def convert_ensg_to_symbol(ensg_list):
    """将 Ensembl Gene ID 列表批量转换为 HUGO Gene Symbol"""
    if not ensg_list:
        return {}
    results = mg.querymany(ensg_list, scopes='ensembl.gene', species='human', fields='symbol', returnall=True)
    mapping = {}
    for hit in results['out']:
        query = hit['query']
        symbol = hit.get('symbol', None)
        mapping[query] = symbol
    return mapping

# 获取 Tahoe 中所有基因名
all_tahoe_genes = adata.var_names.tolist()

# 筛选出 ENSG 格式的基因
ensg_genes = [gene for gene in all_tahoe_genes if gene.startswith('ENSG')]

if ensg_genes:
    print(f"正在将 {len(ensg_genes)} 个 Ensembl Gene ID 转换为 HGNC Symbol...")
    ensg_to_symbol = convert_ensg_to_symbol(ensg_genes)

    # 创建一个完整的从原始基因名到 HGNC Symbol 的映射
    gene_name_mapping = {}
    for gene in all_tahoe_genes:
        if gene.startswith('ENSG'):
            # 如果是ENSG，用转换后的symbol，如果转换失败则保留原ID或设为None
            gene_name_mapping[gene] = ensg_to_symbol.get(gene, None)
        else:
            # 如果不是ENSG，假设它已经是symbol
            gene_name_mapping[gene] = gene

    # 应用映射：创建一个新的列 'hgnc_symbol'
    adata.var['hgnc_symbol'] = adata.var_names.map(gene_name_mapping)

    # 移除映射失败的基因 (即 hgnc_symbol 为 None 的行)
    valid_mask = adata.var['hgnc_symbol'].notnull()
    print(f"移除 {len(adata.var) - valid_mask.sum()} 个无法映射的基因。")
    # 注意：这里我们只是标记，实际计算时再筛选
else:
    print("Tahoe 数据中未发现 Ensembl Gene ID。")
    adata.var['hgnc_symbol'] = adata.var_names

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


正在将 20930 个 Ensembl Gene ID 转换为 HGNC Symbol...


26 input query terms found dup hits:	[('ENSG00000215156', 2), ('ENSG00000226506', 2), ('ENSG00000228044', 2), ('ENSG00000228566', 2), ('E
1078 input query terms found no hit:	['ENSG00000131484', 'ENSG00000132832', 'ENSG00000188525', 'ENSG00000198106', 'ENSG00000203520', 'ENS


移除 16890 个无法映射的基因。


In [38]:
ppi_df

,protein1,protein2,combined_score,gene_a,gene_b
85,9606.ENSP00000000233,9606.ENSP00000158762,825,ARF5,ACAP1
130,9606.ENSP00000000233,9606.ENSP00000357048,718,ARF5,COPA
160,9606.ENSP00000000233,9606.ENSP00000262305,952,ARF5,RAB11FIP3
197,9606.ENSP00000000233,9606.ENSP00000329419,752,ARF5,COPB2
268,9606.ENSP00000000233,9606.ENSP00000469035,795,ARF5,COPE
...,...,...,...,...,...
13715019,9606.ENSP00000501277,9606.ENSP00000326630,942,LDB1,ZFPM1
13715034,9606.ENSP00000501277,9606.ENSP00000263726,944,LDB1,LHX4
13715072,9606.ENSP00000501317,9606.ENSP00000290524,780,RFX7,RFX5
13715299,9606.ENSP00000501317,9606.ENSP00000305071,978,RFX7,RFXANK


In [39]:
# 获取STRING PPI网络中的所有唯一基因 (Symbol)
string_genes = set(ppi_df['gene_a']).union(set(ppi_df['gene_b']))

# 获取Tahoe中所有有效的HGNC Symbol
tahoe_genes = set(adata.var['hgnc_symbol'].dropna())

# 取交集作为候选基因
candidate_genes = list(string_genes & tahoe_genes)

print(f"STRING 网络基因总数: {len(string_genes)}")
print(f"Tahoe 有效基因总数: {len(tahoe_genes)}")
print(f"候选基因 (交集): {len(candidate_genes)}")

# 可选：保存候选基因列表供后续使用
pd.Series(candidate_genes, name='hgnc_symbol').to_csv('candidate_genes.csv', index=False)

STRING 网络基因总数: 16015
Tahoe 有效基因总数: 45686
候选基因 (交集): 15849


In [44]:
def convert_symbol_to_entrez(symbol_list):
    """将 HUGO Gene Symbol 列表批量转换为 Entrez Gene ID"""
    if not symbol_list:
        return {}
    results = mg.querymany(symbol_list, scopes='symbol', species='human', fields='entrezgene', returnall=True)
    mapping = {}
    for hit in results['out']:
        query = hit['query']
        entrez_id = hit.get('entrezgene', None)
        mapping[query] = entrez_id
    return mapping

print("正在将候选基因的 HGNC Symbol 映射到 Entrez Gene ID...")
symbol_to_entrez = convert_symbol_to_entrez(candidate_genes)

# 创建最终的候选基因Entrez ID列表
candidate_entrez_ids = [eid for eid in symbol_to_entrez.values() if eid is not None]
print(f"成功映射到 Entrez ID 的候选基因数量: {len(candidate_entrez_ids)}")

# 创建一个从 Symbol 到 Entrez ID 的映射字典，供后续分析使用
final_gene_mapping = {sym: eid for sym, eid in symbol_to_entrez.items() if eid is not None}

Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed


正在将候选基因的 HGNC Symbol 映射到 Entrez Gene ID...


19 input query terms found dup hits:	[('MMD2', 2), ('PRKY', 2), ('TRIM16L', 2), ('CAST', 2), ('GLRA4', 2), ('CDR1', 2), ('IGKV1-33', 2), 
105 input query terms found no hit:	['CCDC96', 'MRPS36', 'C19orf54', 'GK3P', 'TMEM262', 'TMEM199', 'HHLA3', 'FAM104A', 'GPR182', 'BMT2',


成功映射到 Entrez ID 的候选基因数量: 15737


In [ ]:
candidate_entrez_ids

In [48]:


# ----------------------------
# Step 3: 核心 - 为每个细胞系识别“稳健激活基因”
# ----------------------------

def identify_robust_marker_genes(adata, target_cell_line, candidate_symbols, n_iterations=50, min_presence=0.9, pval_threshold=0.01):
    """
    使用 Wilcoxon 秩和检验 + Bootstrap 重复采样，识别在目标细胞系中稳健激活的基因。

    Args:
        adata: AnnData 对象
        target_cell_line: 目标细胞系名称
        n_iterations: Bootstrap 重复次数
        min_presence: 基因必须在至少多少比例的迭代中被识别为显著，才被认为是“稳健”的
        pval_threshold: Wilcoxon 检验的p值阈值

    Returns:
        stable_genes: 稳健激活基因列表
    """
    gene_names = candidate_symbols
    n_genes = len(gene_names)

    # 存储每次迭代中被识别为显著的基因索引
    significant_gene_indices_per_iter = []

    for i in tqdm(range(n_iterations)):
        # 对目标细胞系进行下采样 (Bootstrap)
        target_cells = adata.obs[adata.obs['cell_name'] == target_cell_line].index.tolist()
        if len(target_cells) < 10: # 如果细胞太少，跳过或报错
            print(f"Warning: Too few cells ({len(target_cells)}) for {target_cell_line}. Skipping robust marker identification.")
            return []

        # Bootstrap: 从目标细胞系中随机有放回地抽取与原始数量相同的细胞
        boot_target_cells = random.choices(target_cells, k=len(target_cells))

        # 非目标细胞系作为背景
        background_cells = adata.obs[adata.obs['cell_name'] != target_cell_line].index.tolist()
        if len(background_cells) == 0:
            print(f"Warning: No background cells found for {target_cell_line}.")
            return []

        # 对每个基因进行 Wilcoxon 秩和检验
        significant_indices = []
        for gene_idx in tqdm(range(n_genes)):
            target_expr = adata[boot_target_cells, gene_idx].X.toarray().flatten()
            background_expr = adata[background_cells, gene_idx].X.toarray().flatten()

            # 执行 Wilcoxon 秩和检验
            try:
                stat, pval = ranksums(target_expr, background_expr, alternative='greater')
                if pval < pval_threshold:
                    significant_indices.append(gene_idx)
            except:
                # 如果检验失败（如所有值相等），跳过
                continue

        significant_gene_indices_per_iter.append(set(significant_indices))

    # 计算每个基因在多少次迭代中被识别为显著
    gene_appearance_count = np.zeros(n_genes, dtype=int)
    for sig_set in significant_gene_indices_per_iter:
        for idx in sig_set:
            gene_appearance_count[idx] += 1

    # 筛选“稳健”基因：出现频率 >= min_presence
    min_appearances = int(n_iterations * min_presence)
    stable_gene_indices = np.where(gene_appearance_count >= min_appearances)[0]
    stable_genes = [gene_names[i] for i in stable_gene_indices]

    print(f"Cell Line '{target_cell_line}': Found {len(stable_genes)} robust marker genes (appeared in >= {min_appearances}/{n_iterations} iterations).")

    stable_genes_entrez = [final_gene_mapping.get(sym) for sym in stable_genes if sym in final_gene_mapping]
    return stable_genes_entrez # 返回 Entrez ID 列表，与 PrimeKG 对齐





In [49]:
# 在主循环前，预先计算每个细胞系的细胞数
cell_line_counts = adata.obs['cell_name'].value_counts()
# 只处理细胞数 >= 10 的细胞系
valid_cell_lines = cell_line_counts[cell_line_counts >= 10].index.tolist()

print(f"细胞数 >= 10 的细胞系有{len(valid_cell_lines)}个")


# 为每个细胞系识别稳健激活基因
robust_marker_genes = {}

def process_single_cell_line(cell_line):
    """处理单个细胞系的函数"""
    print(f"--- Processing Cell Line: {cell_line} ---")
    markers = identify_robust_marker_genes(
        adata,
        target_cell_line=cell_line,
        candidate_symbols=candidate_genes,
        n_iterations=1,
        min_presence=0.9,
        pval_threshold=0.01
    )
    return cell_line, markers

# 并行执行
results = Parallel(n_jobs=-1)(
    delayed(process_single_cell_line)(cl) for cl in valid_cell_lines
)




细胞数 >= 10 的细胞系有44个


{}

In [ ]:
robust_marker_genes = dict(results)
robust_marker_genes

In [52]:
# ----------------------------
# Step 4: 构建protein-protein边
# ----------------------------

# 筛选出两端基因都在候选基因列表中的边
df_p2p = ppi_df[
    (ppi_df['gene_a'].isin(candidate_genes)) &
    (ppi_df['gene_b'].isin(candidate_genes))
].copy()

# 重命名列，使其语义更清晰
df_p2p = df_p2p.rename(columns={'gene_a': 'source', 'gene_b': 'target', 'combined_score': 'weight'})

# 选择最终要导出的列
df_p2p = df_p2p[['source', 'target', 'weight']]

# 导出到 CSV
df_p2p.to_csv('protein2protein.csv', index=False)
print(f"Protein-Protein edges saved to 'protein2protein.csv' ({len(df_p2p)} edges)")

# 查看示例
print("\n--- Sample from protein2protein.csv ---")
print(df_p2p.head())

Protein-Protein edges saved to 'protein2protein.csv' (461922 edges)

--- Sample from protein2protein.csv ---
    source     target  weight
85    ARF5      ACAP1     825
130   ARF5       COPA     718
160   ARF5  RAB11FIP3     952
197   ARF5      COPB2     752
268   ARF5       COPE     795


In [53]:
# Step 5: 构建 Protein-Cell Line 边

# 创建一个空列表来存储所有的边
p2cl_edges = []

# 遍历每个细胞系
for cell_line, expr_series in cell_line_expression.items():
    # 筛选出该细胞系中属于候选基因的表达量
    # 注意：expr_series 的 index 是基因名 (HGNC Symbol)
    filtered_expr = expr_series[expr_series.index.isin(candidate_genes)]

    # 将其转换为 DataFrame 并重命名
    df_temp = filtered_expr.reset_index()
    df_temp.columns = ['source', 'weight'] # 'source' 是基因名
    df_temp['target'] = cell_line # 'target' 是细胞系名

    # 只保留需要的列，并添加到总列表中
    df_temp = df_temp[['source', 'target', 'weight']]
    p2cl_edges.append(df_temp)

# 合并所有细胞系的数据
df_p2cl = pd.concat(p2cl_edges, ignore_index=True)

# 导出到 CSV
df_p2cl.to_csv('protein2cellline.csv', index=False)
print(f"Protein-Cell Line edges saved to 'protein2cellline.csv' ({len(df_p2cl)} edges)")

# 查看示例
print("\n--- Sample from protein2cellline.csv ---")
print(df_p2cl.head())

Protein-Cell Line edges saved to 'protein2cellline.csv' (792450 edges)

--- Sample from protein2cellline.csv ---
     source  target    weight
0  HSP90AA1  SHP-77  3.272727
1    PDE10A  SHP-77  3.090909
2    MT-ND4  SHP-77  2.727273
3      GPC6  SHP-77  2.545455
4    MT-ND5  SHP-77  2.181818


In [54]:
# 验证 Protein-Protein 边
print(f"\nProtein-Protein Network:")
print(f"  - Unique Proteins: {df_p2p['source'].nunique() + df_p2p['target'].nunique() - len(set(df_p2p['source']) & set(df_p2p['target']))}")
print(f"  - Total Edges: {len(df_p2p)}")

# 验证 Protein-Cell Line 边
print(f"\nProtein-Cell Line Network:")
print(f"  - Unique Proteins: {df_p2cl['source'].nunique()}")
print(f"  - Unique Cell Lines: {df_p2cl['target'].nunique()}")
print(f"  - Total Edges: {len(df_p2cl)}")

# 检查是否有空值
print(f"\nAny NaN in P2P? {df_p2p.isnull().any().any()}")
print(f"Any NaN in P2CL? {df_p2cl.isnull().any().any()}")


Protein-Protein Network:
  - Unique Proteins: 15837
  - Total Edges: 461922

Protein-Cell Line Network:
  - Unique Proteins: 15849
  - Unique Cell Lines: 50
  - Total Edges: 792450

Any NaN in P2P? False
Any NaN in P2CL? False
